In [1]:
import numpy as np
import json

In [85]:
def write_group(f, members, name):
    # tiling code needs to be updated for more than 2D
    result = {'name': name, 'size': len(members), 'members': []}
    
    has_identity = False
    for m in members:
        # skip extra identities
        if np.allclose(np.eye(m.shape[0]), m):
            if not has_identity:
                has_identity = True
            else:
                continue
        if not has_identity:
            raise ValueError('Must start with identity')
        i = np.linalg.inv(m)
        # tiling meaning inner Ndim x Ndim is diag only and last column is non-zero
        diag = np.isclose(np.sum(m[:-1, :-1]**2), np.trace(m[:-1, :-1]**2))
        tiling = bool(not np.isclose(np.sum(m[:-1,-1]**2), 0) and diag)
        result['members'].append({'g': list(np.round(m.flatten(), 8)), 'i': list(np.round(i.flatten(), 8)), 't': tiling})
    # sort them - identity first and tiling last    
    result['members'].sort(
        key = lambda m: int(m['t']))
    json.dump(result, f, indent=True)
    print('Wrote group with', len(members), 'members')
    
def cgroup_2d(n):
    thetas = np.linspace(0, 2 * np.pi, n + 1)[:-1]
    r = []
    for t in thetas:
        print(t)
        s, c = np.sin(t), np.cos(t)
        r.append(np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]]))
    return r
def sgroup_2d(n):
    thetas = np.arange(0, n) / n
    r = [np.eye(3)]
    for t in thetas:
        s, c = np.sin(2 * np.pi * t), np.cos(2 * np.pi * t)
        r.append(np.array([[c, s, 0], [s, -c, 0], [0, 0, 1]]))
    return r

def trans_2d(theta):
    v = [np.eye(3) for _ in range(9)]
    a = [np.cos(theta), np.sin(theta)]
    b = [1, 0]
    for i in range(3):
        for j in range(3):
            v[i * 3 + j][0, 2] = (i - 1) * a[0] + (j - 1) * b[0]
            v[i * 3 + j][1, 2] = (i - 1) * a[1] + (j - 1) * b[1]
    # put identity at beginning
    del v[1 * 3 + 1]
    v.insert(0, np.eye(3))
    return v

def outer(g1, g2, remove_dups = False):
    r = []
    for i in g1:
        for j in g2:            
            add = True
            if remove_dups:                
                for ri in r:
                    if np.allclose(i @ j, ri):
                        add = False
                        break            
            if add:
                r.append(i @ j)
    return r

In [98]:
with open('group.json', 'w') as f:
    #g = trans_2d(np.radians(90))
    g = outer(trans_2d(np.radians(90)), cgroup_2d(3))
    #g = cgroup_2d(4)
    write_group(f, g, 'C5')


0.0
2.0943951023931953
4.1887902047863905
Wrote group with 27 members


In [99]:
Ntot = 256
Ns = Ntot // len(g)
N = int(np.floor(np.sqrt(Ns)))
Ns = N * N
Ntot = Ns * len(g)
box = 100
with open('start_positions.xyz', 'w') as f:
    p = np.linspace(box * 0.1, box * 0.9, N)
    for i in range(N):
        for j in range(N):
            #x = p[i] - box / 2 + (np.random.uniform() - 0.5) * box * 0.1
            #y = p[j] - box / 2 + (np.random.uniform() - 0.5) * box * 0.1
            x = np.random.uniform() * box
            y = np.random.uniform() * box
            f.write(f'{x} {y}\n')
    for i in range(Ntot - N):
        f.write('0.0 0.0\n')
print(Ns, Ntot, Ntot / len(g))

9 243 9.0
